<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/MFC/MFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MFC

## Mounting Gdrive

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports

In [16]:
import pandas as pd

## Functions

In [98]:
# https://stackoverflow.com/questions/55695479/why-cant-i-sort-a-list-of-dicts-in-3-x-why-do-i-get-typeerror-not-suppor
# https://www.trainingint.com/how-to-find-duplicates-in-a-python-list.html#:~:text=list%20contains%20duplicates-,mylist%20%3D%20%5B5%2C%203%2C%205%2C%202%2C%201,5%20%26%206%20are%20duplicate%20numbers.&text=Alternatively%2C%20we%20can%20create%20a,to%20alert%20us%20of%20duplicates.&text=return%20False-,mylist%20%3D%20%5B5%2C%203%2C%205%2C%202%2C%201,repeated%20number%20in%20the%20list.

from operator import itemgetter
def return_duplicates(mylist=[]):
    duplist = [] # empty list to hold the duplicate elements from the list
    sorted_list = sorted(mylist, key=itemgetter('start'))
    for index, item in enumerate(sorted_list):
        # print(item)
        try:
            if(item['code'] == sorted_list[index+1]['code'] 
               and (item['start'] == sorted_list[index+1]['start'] or item['start']+1 == sorted_list[index+1]['start']) 
               and (item['end'] == sorted_list[index+1]['end'] or item['end']+1 == sorted_list[index+1]['end'])
               ):
                if item not in duplist:
                    duplist.append(item)
        except IndexError:
            pass      
    return duplist

def adding_duplicate_annotationsDF(df):

    workingDF = df[['annotations','headline_frame','primary_frame', 'text']].copy()
    workingDF.insert(loc=4,  column='duplicates_annotations', value="")

    for index, x in enumerate(tab_df['annotations']):
        annotator = x['framing']
        dupli = []
        # print('r', annotator)
        if len(annotator) != 0:
            for a in annotator:
                # print(a)
                for c in x['framing'][a]:
                    # print(c)
                    dupli.append(c)
            # print(return_duplicates(dupli))
            workingDF.iat[index, 4]= return_duplicates(dupli)
        # else:
            
    workingDF.dropna(
        axis=0,
        how='any',
        inplace=True
    )

    return workingDF
    

In [146]:
# Problem with PRIMARY
# uniforming the codes

def connect_annotation_code(df):
    new_df = pd.DataFrame(columns=['code_frames', 'annotations'])

    for index, row in enumerate(df['duplicates_annotations']):
        for n in row:
            code_frame = n['code']//1
            text =  (df['text'].iloc[index][n['start']:n['end']])
            if (text not in 'PRIMARY'):
                new_row = pd.DataFrame({'code_frames': code_frame, 'annotations':text}, index=[0])
                new_df = pd.concat([new_df.loc[:],new_row]).reset_index(drop=True)
    return new_df

In [147]:
def produce_dataset(df):
    return connect_annotation_code(adding_duplicate_annotationsDF(df))

## DATASETS

### Immigration

In [148]:
IMM_df = pd.read_json(f'{dir}mfc_v4.0/immigration/immigration_labeled.json')
IMM_df = IMM_df.T
IMM_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
Immigration1.0-10005,"{'framing': {'hannah_47.0_r': [{'code': 10.2, ...",10.1,0,10.2,17,IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...,2001,8,15,atlanta journal and constitution
Immigration1.0-10006,"{'framing': {'alex_54.0_r': [{'code': 5.2, 'en...",15.1,0,5.2,19,"IMM-10006\n\nPRIMARY\n\nReaction to Tancredo, ...",2001,8,15,denver post
Immigration1.0-10013,"{'framing': {'jeremy_44.0_r': [{'code': 2.2, '...",1.1,0,2.2,19,"IMM-10013\n\nPRIMARY\n\n$50,000 per entry\n\nT...",2001,8,14,herald-sun
Immigration1.0-10015,"{'framing': {'corey_55.0_r': [{'code': 10.2, '...",None,0,10.2,17,IMM-10015\n\nPRIMARY\n\nDangerous crossing; PU...,2001,8,13,atlanta journal and constitution
Immigration1.0-10018,"{'framing': {'haley_46.0_r': [{'code': 11.2, '...",None,0,None,None,IMM-10018\n\nPRIMARY\n\nCHILDREN OF LATINO IMM...,2001,8,13,san jose mercury news
...,...,...,...,...,...,...,...,...,...,...
Immigration1.0-9981,"{'framing': {'allie_54.0_r': [{'code': 13.2, '...",13.1,0,13.2,17,IMM-9981\n\nPRIMARY\n\nBush Goes Slow on Immig...,2001,8,20,washington post
Immigration1.0-9983,"{'framing': {'haley_49.0_r': [{'code': 2.2, 'e...",None,0,2.2,19,IMM-9983\n\nPRIMARY\n\nTHE OPEN FORUM\n\nLAMM ...,2001,8,19,denver post
Immigration1.0-9986,"{'framing': {'corey_43.0_r': [{'code': 12.2, '...",None,0,12.2,19,IMM-9986\n\nPRIMARY\n\nImmigration Foes Find P...,2001,8,19,washington post
Immigration1.0-9987,"{'framing': {'corey_54.0_r': [{'code': 13.2, '...",None,0,13.2,17,IMM-9987\n\nPRIMARY\n\nEasy immigration finds ...,2001,8,18,atlanta journal and constitution


In [149]:
annoteDS_im = produce_dataset(IMM_df)
annoteDS_im

,code_frames,annotations
0,1.0,\n\nPRIMA
1,14.0,\n\nImmigrants without HOPE need help e
2,1.0,ering college\n\nAnxiety gripped Jose Alvarado...
3,1.0,d a 3.5 grade-point average and 1110 on the SA...
4,6.0,ost Georgia colleges would charge him out-of-s...
...,...,...
4773,5.0,"but we must regain control over who enters, a..."
4774,5.0,\nPRIMAR
4775,15.0,\nFlowers: You ar
4776,9.0,\nPRIMAR


### Gun Control

In [150]:
GC_df = pd.read_json(f'{dir}mfc_v4.0/guncontrol/guncontrol_labeled.json')
GC_df = GC_df.T
GC_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
gun_control1.0-10,"{'framing': {}, 'irrelevant': {'hannah_122.0_r...",None,1,None,None,GUN-10\n\nPRIMARY\n\nNJ argues before NJ Sup C...,1975,5,13,new york times
gun_control1.0-10007,"{'framing': {'marc_134.0_r': [{'code': 13.2, '...",7.1,0,13.2,19,"GUN-10007\n\nPRIMARY\n\nCHANGE THE LAWS, PROTE...",2005,12,21,daily news
gun_control1.0-10008,"{'framing': {'gareth_117.0_r': [{'code': 7.2, ...",6.1,0,7.2,19,GUN-10008\n\nPRIMARY\n\nTIME FOR ALBANY TO GET...,2005,12,18,daily news
gun_control1.0-10013,"{'framing': {'acacia_133.0_r': [{'code': 5.2, ...",5.1,0,5.2,18,GUN-10013\n\nPRIMARY\n\nMan found not guilty i...,2005,12,16,denver post
gun_control1.0-10014,"{'framing': {'allie_134.0_r': [{'code': 13.2, ...",7.1,0,13.2,18,GUN-10014\n\nPRIMARY\n\nPolice Deaths And Stal...,2005,12,15,new york times
...,...,...,...,...,...,...,...,...,...,...
gun_control1.0-9985,"{'framing': {'allie_117.0_r': [{'code': 5.2, '...",3.1,0,5.2,19,GUN-9985\n\nPRIMARY\n\nCHURCH WINS ROUND IN GU...,2004,1,14,saint paul pioneer press
gun_control1.0-9988,"{'framing': {'griffin_130.0_r': [{'code': 7.2,...",7.1,0,7.2,None,GUN-9988\n\nPRIMARY\n\nReport Details Crime Gu...,2004,1,13,washington post
gun_control1.0-9989,{'framing': {'brandon_133.0_r': [{'code': 13.2...,6.1,0,13.2,17,"GUN-9989\n\nPRIMARY\n\nOn Gun Control, Dean Ai...",2004,1,9,washington post
gun_control1.0-9990,"{'framing': {'jade_129.0_r': [{'code': 5.2, 'e...",7.1,0,5.2,17,GUN-9990\n\nPRIMARY\n\nEX-SHERIFF'S AIDE FIGHT...,2004,1,8,saint paul pioneer press


In [151]:
annoteDS_gc = produce_dataset(GC_df)
annoteDS_gc

,code_frames,annotations
0,3.0,Man found not guilty in slaying Questio
1,1.0,illing one of his attackers in his front yard ...
2,3.0,"tate Sen. Jim Brandon, who helped craft the la..."
3,3.0,"o the Hill case.\n\n""If the jury made their de..."
4,3.0,penalties for the sale and posse
...,...,...
7112,11.0,Worst Week in Washington\n\nThe assa
7113,5.0,n measures for Democrats running in places lik...
7114,15.0,Two men of like
7115,15.0,Pro-gun forces target key bills\n\nGun-cont


### Death Penalty

In [152]:
DP_df = pd.read_json(f'{dir}mfc_v4.0/deathpenalty/deathpenalty_labeled.json')
DP_df = DP_df.T
DP_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
death_penalty_100,{'framing': {'sabrina_167.recode_r': [{'code':...,13.1,0,13.2,None,DEA-100\n\nPRIMARY\n\nJERSEY'S ASSEMBLY APPROV...,1983,6,24,new york times
death_penalty_10005,"{'framing': {}, 'irrelevant': {'marc_163.2_r':...",None,1,None,None,DEA-10005\n\nPRIMARY\n\nDEATH SENTENCE IN DEPU...,1997,6,14,san jose mercury news
death_penalty_10006,"{'framing': {'marc_171.2_r': [{'code': 10.2, '...",7.1,0,10.2,17,DEA-10006\n\nPRIMARY\n\n2 MEN ORDERED TO DIE S...,1997,6,14,san jose mercury news
death_penalty_10013,{'framing': {'jade_168.recode_r': [{'code': 8....,8.1,0,8.2,17,DEA-10013\n\nPRIMARY\n\nREPRISAL HINT AT MCVEI...,1997,6,13,daily news
death_penalty_10017,{'framing': {'jade_163.recode_r': [{'code': 5....,None,0,5.2,None,"DEA-10017\n\nPRIMARY\n\n'Cruel, unusual' uncle...",1997,6,13,denver post
...,...,...,...,...,...,...,...,...,...,...
death_penalty_9964,{'framing': {'priscilla_168.recode_r': [{'code...,15.1,0,4.2,19,DEA-9964\n\nPRIMARY\n\nJune 8-14; Open-Handed ...,1997,6,15,new york times
death_penalty_9967,{'framing': {'kaleigh_170.recode_r': [{'code':...,5.1,0,5.2,18,DEA-9967\n\nPRIMARY\n\nDEATH OR LIFE: PENALTY ...,1997,6,15,philadelphia inquirer
death_penalty_9969,"{'framing': {}, 'irrelevant': {'sabrina_167.re...",None,1,None,None,DEA-9969\n\nPRIMARY\n\nAS THEY SEE IT\n\n'It's...,1997,6,15,san jose mercury news
death_penalty_9975,"{'framing': {}, 'irrelevant': {}, 'tone': {'ma...",None,None,None,18,DEA-9975\n\nPRIMARY\n\nTIMOTHY McVEIGH SENTENC...,1997,6,14,atlanta journal and constitution


In [153]:
annoteDS_dp = produce_dataset(DP_df)
annoteDS_dp

,code_frames,annotations
0,14.0,JERSEY'S ASSEMBLY APPROVES DEATH-BY-I
1,1.0,ECTION MEASURE\n\nAfter more than an hour of e...
2,1.0,barbiturate in combination with a chemical pa...
3,6.0,"ve action, the Senate gave initial approval to..."
4,3.0,REPRISAL HINT AT MCVEIGH TRIAL\n\nMoments
...,...,...
4985,1.0,"\nA toddler was abused since birth, and offici..."
4986,1.0,then silence.\n\nThe abuse Glenara faced like...
4987,15.0,May 8-14: 3 Exec
4988,9.0,\nPRIMAR


### Same Sex

In [ ]:
SS_df = pd.read_json(f'{dir}mfc_v4.0/samesex/samesex_labeled.json')
SS_df = SS_df.T
SS_df

In [ ]:
annoteDS_ss = produce_dataset(SS_df)
annoteDS_ss

### Tobacco

In [ ]:
tab_df = pd.read_json(f'{dir}mfc_v4.0/tobacco/tobacco_labeled.json')
tab_df = tab_df.T
tab_df

In [ ]:
annoteDS_tab = produce_dataset(tab_df)
annoteDS_tab

#Try

In [ ]:
tab_dup_df = adding_duplicate_annotationsDF(tab_df)
# tab_dup_df[['text','duplicates_annotations']]
tab_dup_df

In [ ]:
# # char
# l = []
# for index, x in enumerate(st):
#     cod = ""
#     if index >= 57 and index <227:
#         # cod += x
#         l.append(x)
        
# print(l)
    
tab_dup_df['text'].iloc[0][9:16]

## here

In [ ]:
dataset_tab= connect_annotation_code(tab_dup_df)
dataset_tab

In [ ]:
dt_tab = produce_dataset(tab_df)
dt_tab

# END

In [ ]:
# 27597 

In [ ]:
ALL_df = pd.concat([annoteDS_im, annoteDS_gc, annoteDS_dp, annoteDS_ss, annoteDS_tab])
ALL_df